### Code to compare with data

In [3]:
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
from astropy.table import Table
from astropy.io import ascii
from astropy.io import fits
from os import listdir
from os.path import isfile, join
import h5py
import scipy
from scipy import optimize as opt
from scipy import *
from numpy import *
from pylab import *
from scipy.integrate import quad
from scipy.integrate import quad
import astropy.io.ascii as at
from matplotlib.lines import Line2D
import matplotlib
import matplotlib.figure as fig
from matplotlib import rc

G=6.67259e-20 #km³/kg/s²
Msol=1.989e30 #kg
c=299792.458  #km/s
Mpc=3.086e19  #km
H0=72      #km/s/Mpc
M_SNIa=-19.05 #mag

######################################################################
#Defining cosmological functions

DH=c/H0

#Hubble Parameter
def E(z,omegaM,omegaL):
 return sqrt(omegaM*pow(1+z,3)+omegaL)

#Useful integral to estimate distances
def integrand(z,omegaM,omegaL):
 return DH/E(z,omegaM,omegaL)

#Comovel distance
def Dc(z,omegaM,omegaL):
 aux=quad(integrand, 0, z, args=(omegaM,omegaL))
 aux=aux[0]
 return aux

#Angular diameter distance
def DA(z,omegaM,omegaL):
 aux=quad(integrand, 0, z, args=(omegaM,omegaL))
 aux=aux[0]
 return aux/(1+z)

#Function can receive vector as argument
dav=vectorize(DA)

#Luminosity distance
def Dl(z,omegaM,omegaL):
 aux=quad(integrand, 0, z, args=(omegaM,omegaL))
 aux=aux[0]
 return (1+z)*aux

dlv=vectorize(Dl)

#Distance modulus
def DM(z,omegaM,omegaL):
 return 5.0*log10(Dl(z,omegaM,omegaL))+25

DMV=vectorize(DM)

In [4]:
#Reading the ACTPol catalogue
os.chdir('/home/stephanewerner/Documents/pzwav_2021/catalogues_comparison')
matrix = np.loadtxt('actpol_cutradec',skiprows=1, delimiter=',')
matrix = matrix[np.where((matrix[:,2] >0.10) &(matrix[:,2] <0.40) & (matrix[:,1]<1.3) & (matrix[:,1]>-1.3) & (((matrix[:,0]<60.0) & (matrix[:,0]>0.0)) | ((matrix[:,0]<360.0) & (matrix[:,0]>315.0) ) ))]
ra_cls = matrix[:,0]
dec_cls = matrix[:,1]
z_cls = matrix[:,2]
logM200_cls = matrix[:,3]
#error_mass1 =matrix[:,5]
#error_mass2 = matrix[:,6]
#error_mass = matrix[:,5] + matrix[:,6]

matrix = matrix[np.where(matrix[:,2] <0.40)]
ra_cls = matrix[:,0]
dec_cls = matrix[:,1]
z_cls = matrix[:,2]
logM200_cls = matrix[:,3]

In [12]:
#Reading X-RAY CATALOGUE
#os.chdir('/home/stephane/Documents/catalog_cluster2019/2019')
matrix = np.loadtxt('3XMMSDSS_selected',skiprows=1, delimiter=',', usecols=(1,2,3))
matrix = matrix[np.where((matrix[:,2] >0.10) & (matrix[:,2] <0.40) & (matrix[:,1]<1.3) & (matrix[:,1]>-1.3) & (((matrix[:,0]<60.0) & (matrix[:,0]>0.0)) | ((matrix[:,0]<360.0) & (matrix[:,0]>315.0) ) ))]
ra_cls = matrix[:,0]
dec_cls = matrix[:,1]
z_cls = matrix[:,2]

In [41]:
#Reading RedMapper
#os.chdir('/home/stephane/Documents/catalog_cluster2019/2019')
matrix = np.loadtxt('redmapper_read',skiprows=1, delimiter=',')
matrix = matrix[np.where((matrix[:,4] >0.1)&(matrix[:,4] <0.40) & (matrix[:,3]<1.3) & (matrix[:,3]>-1.3) & (((matrix[:,2]<60.0) & (matrix[:,2]>0.0)) | ((matrix[:,2]<360.0) & (matrix[:,2]>315.0) ) ))]
ra_cls = matrix[:,2]
dec_cls = matrix[:,3]
z_cls = matrix[:,4]

In [20]:
#Reading XCSDR1_Mehrtens
#os.chdir('/home/stephane/Documents/catalog_cluster2019/2019')
matrix = np.loadtxt('XCSDR1withoutname', delimiter=',', skiprows=1)
matrix = matrix[np.where((matrix[:,4] >0.10)&(matrix[:,4] <0.40) & (matrix[:,1]<1.30) & (matrix[:,1]>-1.30) & (((matrix[:,0]<60.0) & (matrix[:,0]>0.0)) | ((matrix[:,0]<360.0) & (matrix[:,0]>315.0) ) ))]
ra_cls = matrix[:,0]
dec_cls = matrix[:,1]
z_cls = matrix[:,4]

In [46]:
#Reading GEACH 2011 GMB
#os.chdir('/home/stephane/Documents/catalog_cluster2019/2019')
#Cluster;RAJ2000;DEJ2000;RA1deg;DE1deg;Ngal;z;r80;R
matrix = np.loadtxt('geach2011_Stripe.tsv', delimiter=';', usecols = (1,2,6), skiprows=1)
matrix = matrix[np.where((matrix[:,2] >0.10) & (matrix[:,2] <0.40) & (matrix[:,1]<1.20) & (matrix[:,1]>-1.20) & (((matrix[:,0]<60.0) & (matrix[:,0]>0.0)) | ((matrix[:,0]<360.0) & (matrix[:,0]>315.0) ) ))]
ra_cls = matrix[:,0]
dec_cls = matrix[:,1]
z_cls = matrix[:,2]


In [32]:
#Reading Wen
#os.chdir('/home/stephane/Documents/catalog_cluster2019/2019')
#Cluster;RAJ2000;DEJ2000;RA1deg;DE1deg;Ngal;z;r80;R
matrix = np.loadtxt('catalog_clusters_wen.csv', delimiter=',', usecols = (0,1,5), skiprows=1)
matrix = matrix[np.where((matrix[:,2]!= -1.) & (matrix[:,2] <0.40) & (matrix[:,2] >0.1)& (matrix[:,1]<1.3) & (matrix[:,1]>-1.3) & (((matrix[:,0]<60.0) & (matrix[:,0]>0.0)) | ((matrix[:,0]<360.0) & (matrix[:,0]>315.0) ) ))]
ra_cls = matrix[:,0]
dec_cls = matrix[:,1]
z_cls = matrix[:,2]

In [51]:
#Durret
#os.chdir('/home/stephane/Documents/catalog_cluster2019/2019')
#Seq;RAJ2000;DEJ2000;z;Ngal
matrix = np.loadtxt('durret.tsv', delimiter=';', usecols = (1,2,3), skiprows=1)
matrix = matrix[np.where((matrix[:,2] >0.10) &(matrix[:,2] <0.40) & (matrix[:,1]<1.3) & (matrix[:,1]>-1.3) & (((matrix[:,0]<60.0) & (matrix[:,0]>0.0)) | ((matrix[:,0]<360.0) & (matrix[:,0]>315.0) ) ))]
ra_cls = matrix[:,0]
dec_cls = matrix[:,1]
z_cls = matrix[:,2]

In [52]:
hdulist = fits.open('detclPZWAV_CFC4M2A302.80H21SN0.2V2.1.fits')
scidata = hdulist[1].data
prihdr = hdulist[0].header
sechdr= hdulist[1].header

ra_det=[]
dec_det =[]
z_det =[]
id_det=[]
riq_det =[]
rank_det =[]
zerr_det=[]
sn_det=[]

for i in range(len(scidata)):
    ra_det.append(scidata[i][1])
    dec_det.append(scidata[i][2])
    z_det.append(scidata[i][3])
    zerr_det.append(scidata[i][4])
    id_det.append(scidata[i][0])
    riq_det.append(scidata[i][6])
    rank_det.append(scidata[i][8])
    sn_det.append(scidata[i][5])

ra_det = np.array(ra_det)
dec_det = np.array(dec_det)
z_det = np.array(z_det)
id_det = np.array(id_det)
riq_det = np.array(riq_det)
rank_det = np.array(rank_det)
zerr_det = np.array(zerr_det)
sn_det = np.array(sn_det)

table = np.zeros((len(rank_det), 8))
table[:,0] = ra_det
table[:,1] = dec_det
table[:,2] = z_det
table[:,3] = id_det
table[:,4] = riq_det
table[:,5] = rank_det
table[:,6] = zerr_det
table[:,7] = sn_det

table = table[np.where((table[:,7] > 4.0) &  (table[:,2] < 0.4) )]


for i in range(len(table)):
    table[i,0] = table[i,0] + 220
    if table[i,0] > 360.0:
        table[i,0]= table[i,0]-360.0
    else:
        table[i,0] = table[i,0]

ra_det = table[:,0]
dec_det = table[:,1]
z_det =table[:,2]
id_det = table[:,3]
riq_det = table[:,4]
rank_det= table[:,5]
zerr_det = table[:,6]
sn_det= table[:,7]

In [30]:
#Fazendo match

#Doing the Geometrical Matching (Step 1)
M200 = logM200_cls
ID_det =[]
ID_mock=[]
riqueza =[]
m200 =[]
rank = []
dist = []
z_spec1 =[]
zdet =[]
zerrdet =[]
RA_det =[]
DEC_det =[]
Ra_cls=[]
Dec_cls=[]
SN_det =[]
ra_non =[]
dec_non =[]
z_non =[]
mass =[]
riqueza =[]
distancia = []
zdeterminado =[]
#Parameters that we use: 1Mpc and delta_z=0.3
import tqdm

print('tamanho do catalogo:',len(ra_cls))

for j in tqdm.tqdm(range(len(ra_cls))):
    #print(j, 'redshift:',z_cls[j])
    dist = []
    z1= []
    print(j)
    for k in range(len(ra_det)):
        a= np.sin(np.deg2rad(dec_det[k]))*np.sin(np.deg2rad(dec_cls[j]))
        b = np.cos(np.deg2rad(dec_det[k]))*np.cos(np.deg2rad(dec_cls[j]))
        c = np.cos(np.deg2rad(ra_det[k]))*np.cos(np.deg2rad(ra_cls[j])) + np.sin(np.deg2rad(ra_det[k]))* np.sin(np.deg2rad(ra_cls[j]))
        if (((DA(z_cls[j],0.3,0.7)*(np.arccos(a + b*c)))<1.0) & (np.absolute(z_det[k] - z_cls[j]) < 0.05)):
            dist.append((DA(z_cls[j],0.3,0.7)*(np.arccos(a + b*c))))
            z1.append(z_cls[j])
    #Queremos agora só o que tem menor distancia como o centro
    if dist != []:
        index = np.where(min(np.array(dist)))
        z1 = np.array(z1)
        z_spec1.append(z1[index])
        print('yes')

  0%|          | 0/762 [00:00<?, ?it/s]

tamanho do catalogo: 762
0


  0%|          | 1/762 [00:00<07:27,  1.70it/s]

yes
1


  0%|          | 2/762 [00:01<07:20,  1.72it/s]

...

100%|█████████▉| 760/762 [07:13<00:01,  1.77it/s]

760


100%|█████████▉| 761/762 [07:13<00:00,  1.75it/s]

761


100%|██████████| 762/762 [07:14<00:00,  1.75it/s]


In [31]:
#porcentagem 

percentage = (len(z_spec1)/len(ra_cls))*100
print(percentage)

26.77165354330709


In [53]:
#Optical analysis (opposite way)


#Doing the Geometrical Matching (Step 1)
M200 = logM200_cls
ID_det =[]
ID_mock=[]
riqueza =[]
m200 =[]
rank = []
dist = []
z_spec1 =[]
zdet =[]
zerrdet =[]
RA_det =[]
DEC_det =[]
Ra_cls=[]
Dec_cls=[]
SN_det =[]
ra_non =[]
dec_non =[]
z_non =[]
mass =[]
riqueza =[]
distancia = []
zdeterminado =[]
#Parameters that we use: 1Mpc and delta_z=0.3
import tqdm

print('tamanho do catalogo:',len(ra_cls))

for j in tqdm.tqdm(range(len(ra_det))):
    #print(j, 'redshift:',z_cls[j])
    dist = []
    z1= []
    print(j)
    for k in range(len(ra_cls)):
        a= np.sin(np.deg2rad(dec_det[j]))*np.sin(np.deg2rad(dec_cls[k]))
        b = np.cos(np.deg2rad(dec_det[j]))*np.cos(np.deg2rad(dec_cls[k]))
        c = np.cos(np.deg2rad(ra_det[j]))*np.cos(np.deg2rad(ra_cls[k])) + np.sin(np.deg2rad(ra_det[j]))* np.sin(np.deg2rad(ra_cls[k]))
        if (((DA(z_cls[k],0.3,0.7)*(np.arccos(a + b*c)))<1.0) & (np.absolute(z_det[j] - z_cls[k]) < 0.05)):
            dist.append((DA(z_cls[k],0.3,0.7)*(np.arccos(a + b*c))))
            z1.append(z_cls[k])
    #Queremos agora só o que tem menor distancia como o centro
    if dist != []:
        index = np.where(min(np.array(dist)))
        z1 = np.array(z1)
        z_spec1.append(z1[index])
        print('yes')



  0%|          | 1/4685 [00:00<08:25,  9.27it/s]

tamanho do catalogo: 762
0
1


  0%|          | 3/4685 [00:00<08:40,  8.99it/s]

2
3


  0%|          | 5/4685 [00:00<08:52,  8.79it/s]

...

4678
4679
4680


100%|█████████▉| 4682/4685 [08:02<00:00, 10.37it/s]

4681
4682
4683


100%|██████████| 4685/4685 [08:02<00:00,  9.70it/s]

4684


In [54]:
#porcentagem 

percentage = (len(z_spec1)/len(ra_det))*100
print(percentage)

4.461045891141943


In [39]:
len(z_spec1)

271

In [40]:
len(ra_det)

4685

In [70]:
#Vamos comparar o quanto esses catálogos do óptico recuperam em raios-X
#Considerando o XMMSDSS

matrix = np.loadtxt('3XMMSDSS_selected',skiprows=1, delimiter=',', usecols=(1,2,3))
matrix = matrix[np.where((matrix[:,2] >0.10) & (matrix[:,2] <0.40) & (matrix[:,1]<1.3) & (matrix[:,1]>-1.3) & (((matrix[:,0]<60.0) & (matrix[:,0]>0.0)) | ((matrix[:,0]<360.0) & (matrix[:,0]>315.0) ) ))]
ra_cls = matrix[:,0]
dec_cls = matrix[:,1]
z_cls = matrix[:,2]

In [56]:
matrix = np.loadtxt('catalog_clusters_wen.csv', delimiter=',', usecols = (0,1,5), skiprows=1)
matrix = matrix[np.where((matrix[:,2]!= -1.) & (matrix[:,2] <0.40) & (matrix[:,2] >0.1)& (matrix[:,1]<1.3) & (matrix[:,1]>-1.3) & (((matrix[:,0]<60.0) & (matrix[:,0]>0.0)) | ((matrix[:,0]<360.0) & (matrix[:,0]>315.0) ) ))]
ra_det = matrix[:,0]
dec_det = matrix[:,1]
z_det = matrix[:,2]

In [59]:
matrix = np.loadtxt('redmapper_read',skiprows=1, delimiter=',')
matrix = matrix[np.where((matrix[:,4] >0.1)&(matrix[:,4] <0.40) & (matrix[:,3]<1.3) & (matrix[:,3]>-1.3) & (((matrix[:,2]<60.0) & (matrix[:,2]>0.0)) | ((matrix[:,2]<360.0) & (matrix[:,2]>315.0) ) ))]
ra_det = matrix[:,2]
dec_det = matrix[:,3]
z_det = matrix[:,4]

In [62]:
matrix = np.loadtxt('geach2011_Stripe.tsv', delimiter=';', usecols = (1,2,6), skiprows=1)
matrix = matrix[np.where((matrix[:,2] >0.10) & (matrix[:,2] <0.40) & (matrix[:,1]<1.20) & (matrix[:,1]>-1.20) & (((matrix[:,0]<60.0) & (matrix[:,0]>0.0)) | ((matrix[:,0]<360.0) & (matrix[:,0]>315.0) ) ))]
ra_det = matrix[:,0]
dec_det = matrix[:,1]
z_det = matrix[:,2]

In [71]:
matrix = np.loadtxt('durret.tsv', delimiter=';', usecols = (1,2,3), skiprows=1)
matrix = matrix[np.where((matrix[:,2] >0.10) &(matrix[:,2] <0.40) & (matrix[:,1]<1.3) & (matrix[:,1]>-1.3) & (((matrix[:,0]<60.0) & (matrix[:,0]>0.0)) | ((matrix[:,0]<360.0) & (matrix[:,0]>315.0) ) ))]
ra_det = matrix[:,0]
dec_det = matrix[:,1]
z_det = matrix[:,2]

In [72]:

#Doing the Geometrical Matching (Step 1)
M200 = logM200_cls
ID_det =[]
ID_mock=[]
riqueza =[]
m200 =[]
rank = []
dist = []
z_spec1 =[]
zdet =[]
zerrdet =[]
RA_det =[]
DEC_det =[]
Ra_cls=[]
Dec_cls=[]
SN_det =[]
ra_non =[]
dec_non =[]
z_non =[]
mass =[]
riqueza =[]
distancia = []
zdeterminado =[]
#Parameters that we use: 1Mpc and delta_z=0.3
import tqdm

print('tamanho do catalogo:',len(ra_cls))

for j in tqdm.tqdm(range(len(ra_cls))):
    #print(j, 'redshift:',z_cls[j])
    dist = []
    z1= []
    print(j)
    for k in range(len(ra_det)):
        a= np.sin(np.deg2rad(dec_det[k]))*np.sin(np.deg2rad(dec_cls[j]))
        b = np.cos(np.deg2rad(dec_det[k]))*np.cos(np.deg2rad(dec_cls[j]))
        c = np.cos(np.deg2rad(ra_det[k]))*np.cos(np.deg2rad(ra_cls[j])) + np.sin(np.deg2rad(ra_det[k]))* np.sin(np.deg2rad(ra_cls[j]))
        if (((DA(z_cls[j],0.3,0.7)*(np.arccos(a + b*c)))<1.0) & (np.absolute(z_det[k] - z_cls[j]) < 0.05)):
            dist.append((DA(z_cls[j],0.3,0.7)*(np.arccos(a + b*c))))
            z1.append(z_cls[j])
    #Queremos agora só o que tem menor distancia como o centro
    if dist != []:
        index = np.where(min(np.array(dist)))
        z1 = np.array(z1)
        z_spec1.append(z1[index])
        print('yes')

  3%|▎         | 1/29 [00:00<00:03,  8.36it/s]

tamanho do catalogo: 29
0
yes
1


 14%|█▍        | 4/29 [00:00<00:02,  8.92it/s]

2
3
4


 21%|██        | 6/29 [00:00<00:02,  9.31it/s]

...

24
25


 93%|█████████▎| 27/29 [00:03<00:00,  8.33it/s]

26
27


100%|██████████| 29/29 [00:03<00:00,  7.43it/s]

28


In [73]:
#porcentagem 

percentage = (len(z_spec1)/len(ra_cls))*100
print(percentage)

6.896551724137931
